In [365]:
# Libraries
from fpdf import FPDF
import pandas as pd
import yaml


In [366]:
# Define the path and filename
yaml_path = "../data/page_layout.yaml"


# Load the YAML file
with open(yaml_path , 'r') as file:
    data = yaml.safe_load(file)
    
title_report = "DOE Summary"


In [367]:
class ReportStrategy:
    def __init__(self, header_img_path, chart_img_paths, output_file):
        self.pdf = FPDF()
        self.header_img_path = header_img_path
        self.chart_img_paths = chart_img_paths
        self.output_file = output_file

    def add_header(self):
        """Add the header image and title to the PDF"""
        self.pdf.add_page()
        self.pdf.image(self.header_img_path, x=0, y=0, w=210)
        self.pdf.set_text_color(255, 255, 255)  # RGB for black
        self.pdf.set_xy(1, 14)
        self.pdf.add_font('Montserrat-Bold', 'B', './assets/Montserrat/static/Montserrat-Bold.ttf', uni=True)
        self.pdf.set_font('Montserrat-Bold', 'B', 24)
        self.pdf.multi_cell(0, 10, f"{title_report}", 0, 'L')
        
    def add_summary_tables(self, titles, numbers):
        """Add three summary tables with a blue header and centered numbers"""
        box_width = 60
        box_height = 35
        blue_header_height = 15
        spacing = 10
        start_x = (210 - (3 * box_width + 2 * spacing)) / 2  # Center the boxes horizontally
        y_position = 45  # Position below the header

        for i, (title, number) in enumerate(zip(titles, numbers)):
            self.pdf.set_font('Montserrat-Bold', 'B', 12)
            x_position = start_x + i * (box_width + spacing)
            
            # Draw the blue header
            self.pdf.set_fill_color(25, 36, 59) # Blue color
            self.pdf.rect(x_position, y_position, box_width, blue_header_height, style='F')
            
            # Add the title text inside the blue header
            self.pdf.set_text_color(255, 255, 255)  # White text
            self.pdf.set_xy(x_position, y_position + 2)
            self.pdf.cell(box_width, blue_header_height - 8, title, 0, 0, 'C')
            
            # Add the subtitle text inside the blue header
            self.pdf.set_font('Montserrat-Bold', 'B', 7)
            self.pdf.set_text_color(255, 255, 255)  # White text
            self.pdf.set_xy(x_position, y_position + 8)
            self.pdf.cell(box_width, blue_header_height - 10, "(Change, M bbl)", 0, 0, 'C')
            
            # Draw the white box below the header
            self.pdf.set_draw_color(0, 0, 0)  # Black border
            self.pdf.set_fill_color(255, 255, 255)  # White fill
            self.pdf.rect(x_position, y_position + blue_header_height, box_width, box_height - blue_header_height, style='FD')
            
            # Add the number below the blue header
    # Set text color based on number value
            if number < 0:
                self.pdf.set_text_color(255, 0, 0)  # Red text for negative values
            else:
                self.pdf.set_text_color(0, 128, 0)  # Dark green text for positive values
            self.pdf.set_font('Montserrat-Bold', 'B', 18)
            self.pdf.set_xy(x_position, y_position + blue_header_height + (box_height - blue_header_height) / 2 - 5)
            self.pdf.cell(box_width, 10, str(number), 0, 0, 'C')

        # Reset font for the rest of the document
        self.pdf.set_font('Montserrat-Bold', 'B', 16)

        
    def add_charts(self):
        """Add the chart images to the PDF in a 2x2 grid layout"""
        page_width = self.pdf.w
        page_height = self.pdf.h
        image_width = 97  # Image width (adjust to fit layout)
        image_height = 74  # Image height (adjust to fit layout)
        horizontal_margin = 5  # Space between images horizontally
        vertical_margin = 5  # Space between images vertically

        # Initial y-position (can be set to any desired value)
        initial_y_position = 90  # Start a bit lower on the page (e.g., 50 units from top)
        current_y_position = initial_y_position

        # Use a list of lists so you can modify the y-values
        positions = [
            [horizontal_margin, current_y_position],  # Top-left
            [horizontal_margin + image_width + horizontal_margin, current_y_position],  # Top-right
            [horizontal_margin, current_y_position + image_height + vertical_margin],  # Bottom-left
            [horizontal_margin + image_width + horizontal_margin, current_y_position + image_height + vertical_margin],  # Bottom-right
        ]

        # Iterate over the chart image paths and place them in the grid
        for i, chart_image_path in enumerate(self.chart_img_paths):
            if i >= 4:  # Stop after 4 images
                break
            
            x_position, y_position = positions[i]
            
            # Add image to the PDF at the calculated position
            self.pdf.image(chart_image_path, x=x_position, y=y_position, w=image_width, h=image_height)
            
            # Update current y-position after placing each image
            if (i + 1) % 2 == 0:  # After placing 2 images in one row, move down for the next row
                current_y_position += image_height + vertical_margin
                positions[2 * (i // 2)][1] = current_y_position  # Update the starting y for the bottom row

        # Move the cursor down after images are placed
        self.pdf.ln(image_height + vertical_margin)
        
    def add_source(self):
        """Add the source to the PDF"""
        from datetime import datetime
        year = datetime.now().year
        
        # Create the copyright text
        copyright_text = f"© {year}. All rights reserved.\n"
        copyright_text += "The data used in this report are sourced from the U.S. Energy Information Administration (EIA)."
        
        # Set font and text properties
        self.pdf.add_font('Montserrat-Bold', 'I', './assets/Montserrat/static/Montserrat-Italic.ttf', uni=True)
        self.pdf.set_font('Montserrat-Bold', 'I', 9)
        self.pdf.set_text_color(0, 0, 0)  # RGB for black
        
        # Set the position to center the text on the page
        self.pdf.set_xy(0, 260)  # Starting X=0 for center alignment
        
        # Add the text in a multi-cell, centered and justified
        self.pdf.multi_cell(0, 5, f"{copyright_text}", 0, 'C')  # 'C' centers the text
        
                    

    def generate_report(self, title_report, titles, numbers):
        """Generate and save the report as a PDF"""
        self.add_header()
        self.add_summary_tables(titles, numbers)
        self.add_charts()
        self.add_source()
        self.pdf.output(self.output_file)

In [368]:
# Usage example:
header_img_path = './assets/header.png'
chart_img_paths = [
    '../report/images/all_petroleum_products.png',
    '../report/images/oil_commercial_stocks.png',
    '../report/images/m_gasoline_stocks.png',
    '../report/images/m_distillate_stocks.png',
]

# Get today's date in the desired format
from datetime import datetime
today_date = datetime.now().strftime("%Y %m %d")

# Generate the output file path with the formatted date
output_file = f'../report/DOE Summary - {today_date}.pdf'

titles = ["Commercial Oil Stocks", "M. Gasoline Stocks", "M. Distillates Stocks"]
numbers = [data["oil_commercial_stocks_chg"], data["m_gasoline_stocks_chg"], data["m_distillate_stocks_chg"]]

report = ReportStrategy(header_img_path, chart_img_paths, output_file)
report.generate_report(title_report, titles, numbers)